<a href="https://colab.research.google.com/github/marianasmoura/tecnicas-de-otimizacao/blob/main/Otimizacao_irrestrita_Davidon_Fletcher_Powell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UNIVERSIDADE FEDERAL DO PIAUÍ

CURSO DE GRADUAÇÃO EM ENGENHARIA ELÉTRICA

DISCIPLINA: TÉCNICAS DE OTIMIZAÇÃO

DOCENTE: ALDIR SILVA SOUSA

DISCENTE: MARIANA DE SOUSA MOURA

Atividade 4: Otimização Irrestrita pelo Método de Devidon-Fletcher-Powell

Resolva os exercícios usando o método de Devidon-Fletcher-Powell.

**A classe Parametros**

Esta classe tem por finalidade enviar em uma única variável, todos os parâmetros necessários para executar o método. 


Para o método de Davidon-Fletcher-Powell, precisamos da função que se deseja minimizar, o intervalo de incerteza inicial e a tolerância requerida.



In [ ]:
import numpy as np
import sympy as sym

class Parametros:
  def __init__(self,f,x0,vars,eps):
    self.f = f
    self.x0 = np.array(x0)
    self.vars = vars
    self.eps = eps

#**Método de Newton Monovariável**

Trecho responsável por realizar a minimização do valor de lamda, que corresponde a taxa com a qual a otimização da função é feita. 

In [ ]:
lmbd = sym.Symbol('lmbd')
def sym_newton(f,x1,eps):
   x  =  x1
   px = x
   k  = 1
   stop = False
   df = sym.diff(f)
   d2f = sym.diff(df)
   map = dict()
   
   while (not stop):
       map[lmbd]= float(x)
       jac = df.subs(map)
       hes = d2f.subs(map)
       px = x
       x = x - float(jac/hes)
       e = abs(px - x)
       k = k + 1  
       if  (e < eps): stop=True
       if (k > 30): stop=True
       if (abs(jac) < eps):stop=True
   return  x


#**Cômputo do gradiente e avaliação das funções nos pontos**

Nesta seção, têm-se definições de funções para calcular os valores do gradiente e da hessiana utilizando-se a função em termos de variáveis simbólicas. Também realiza-se aqui, por meio dessas definições de funções, a substituição das variáveis simbólicas por valores numéricos passados como parâmetro.

In [ ]:
#Avalia a função simbólica f em x
def eval_f_simbolica(f,variaveis,x,convertToFloat=False):
   mp = dict()
   for i in range(len(variaveis)):
      mp[variaveis[i]] = x[i]
   if (convertToFloat):
      return float(f.subs(mp))
   return f.subs(mp)
   #return f.subs(mp)

def eval_f_simb(f,variaveis,x):
   mp = dict()
   for i in range(len(variaveis)):
      mp[variaveis[i]] = x[i]
   return f.subs(mp)

# Retorna o gradiente simbólico
def gradiente_simbolico(funcao,variaveis):
     g = [sym.diff(funcao,x) for x in variaveis]
     return g  

# Retorna o vetor gradiente em x
def eval_gradiente(grad_simb,variaveis,x):
   
   g = [ eval_f_simbolica(f,variaveis,x) for f in grad_simb]
   return np.array(g)


#**O método Davidson-Fletcher-Powell**

Principal definição para o método. Aqui são passados como parâmetro a função, intervalo de incerteza inicial e tolerância, utilizados para realizar a otimização. Dentro desta função calcula-se a derivada e a matriz D2 simétrica resultante (que substitui a matriz hessiana do método de Newton) após diversas iterações em busca do ponto de mínimo. E, após convergência, encontra-se o valor de y, ponto de mínimo para determinada tolerância.

In [ ]:
import pandas as pd
import math

def Davidon_Fletcher_Powell(params):
    n = len(params.x0)
    x = params.x0
    D = np.eye(n,n) # gerando uma matriz identidade nxn
    vars = params.vars
    f = params.f
    eps = params.eps

    cols = ['x','y','df(x)']    
    table = pd.DataFrame([], columns=cols)

    j = k = 0
    y = x

    sym_grad = gradiente_simbolico(f,vars) #criando o vetor gradiante simbólico
    G = eval_gradiente(sym_grad,vars,y) #calcula o gradiente simbólico em y
    G_ant = G
    B = np.array(G)
    tolerancia = pow(sum(pow(B,2)),1/2)
    #Passo 1 do passo principal.
    while (j < n):
          if (tolerancia<eps): #critério de parada ===> np.linalg.norm(G) equivale ao módulo
              return solucao,table
              
          d = -np.matmul(np.array(D),G)  #calculando   -D x G'
          #para calcular min f(y + lambda* d)
          #m = y.flatten() + lmbd*d # versão anterior
          m = y + lmbd*d

          subProb = eval_f_simb(f,vars,m)
          lmbdOtimo = sym_newton(subProb,0,0.001)
          
          y_ant = y
          #y = y.flatten() + lmbdOtimo*d
          y = y + lmbdOtimo*d

          if (j < (n-1)):
            #Passo 2 do passo principal
             G_ant = G
             G = eval_gradiente(sym_grad,vars,y)

             p = y - y_ant
             q = G - G_ant
             q = q.reshape(n,1) 

             pt = p.transpose()
             qt = q.transpose()
             
             p_pt = p.dot(pt)
             pt_q = pt.dot(q)
             

             qt_D = np.matmul(qt,D) 
             D_q = np.matmul(D,q)
             
             num  = np.matmul(D_q,qt_D)
             dem  = np.matmul(qt_D,q)
            
             D = D + p_pt/pt_q - num/dem
                 
          else:
              D = np.eye(n,n) # gerando uma matriz identidade nxn
              j = -1
              x = y
              #print('x: ',x.flatten())
          j = j + 1

          solucao = list(y)
          B = np.array(G)
          tolerancia = pow(sum(pow(B,2)),1/2)
          row = pd.DataFrame(list([[x,y,G]]),columns=cols)
          table = table.append(row, ignore_index=True) # concatena valores de cada iteração
    return solucao,table 
  

**1.** Considere o seguinte problema:

Minimizar $\sum_{i=2}^{n} [100(x_i-x^2_{i-1})^2 + (1-x_{i-1})^2]$

Resolva para n = 5, 10, e 50. Iniciando do ponto $x_0 = [-1.2,1.0,-1.2,1.0,...]$

In [ ]:
# Para n = 5
import numpy as np
import sympy as sym 


variaveis = list(sym.symbols("x:5"))
c = variaveis

def f(c):
  fo = 0
  for i in range(1,5):
     fo = fo + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2 
  return fo

x = []
for i in range(1,6):
    if (i%2 != 0):
        x.append(-1.2)
    else:
        x.append(1)  

eps = 1e-3
p  = Parametros(f(c),x,c,eps)
y,table = Davidon_Fletcher_Powell(p)

In [ ]:
y

[0.999955714246688,
 0.999912026734920,
 0.999824951144613,
 0.999648476789618,
 0.999295698727518]

In [ ]:
table

,x,y,df(x)
0,"[-1.2, 1.0, -1.2, 1.0, -1.2]","[-1.06832397392761, 0.516292149121821, -0.7995...","[-271.227882018711, 94.2067684642814, -256.190..."
1,"[-1.2, 1.0, -1.2, 1.0, -1.2]","[-0.793673731889360, 0.640867373737510, -0.678...","[-0.111253017012160, 280.679602471813, -215.63..."
2,"[-1.2, 1.0, -1.2, 1.0, -1.2]","[-0.854933688861322, 0.642036288692871, -0.540...","[-34.1028706989055, 226.273900679618, -98.3330..."
3,"[-1.2, 1.0, -1.2, 1.0, -1.2]","[-0.947345102043352, 0.696352955493222, -0.643...","[-80.1028388306721, 273.467787065711, -145.789..."
4,"[-0.847182478692355, 0.539057854628998, -0.851...","[-0.847182478692355, 0.539057854628998, -0.851...","[-80.1028388306721, 273.467787065711, -145.789..."
...,...,...,...
147,"[0.999975782934895, 0.999923810190700, 0.99982...","[0.999970972279620, 0.999921598361809, 0.99982...","[0.00808052432370232, 0.00307923166616320, 0.0..."
148,"[0.999975782934895, 0.999923810190700, 0.99982...","[0.999969955136985, 0.999926959881690, 0.99983...","[0.00512027261589765, 0.00599443357663176, 0.0..."
149,"[0.999954219961018, 0.999913924161985, 0.99982...","[0.999954219961018, 0.999913924161985, 0.99982...","[0.00512027261589765, 0.00599443357663176, 0.0..."
150,"[0.999954219961018, 0.999913924161985, 0.99982...","[0.999954360630490, 0.999914088847319, 0.99982...","[-0.00223738214259583, 0.00206784432936047, 3...."


In [ ]:
# Para n = 10
import numpy as np
import sympy as sym 


variaveis = list(sym.symbols("x:10"))

c = variaveis

def f(c):
  fo = 0
  for i in range(1,10):
     fo = fo + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2 
  return fo

x = []
for i in range(1,11):
    if (i%2 != 0):
        x.append(-1.2)
    else:
        x.append(1)  

eps = 1e-3
p  = Parametros(f(c),x,c,eps)
y,table = Davidon_Fletcher_Powell(p)

In [ ]:
y

[-0.993262625748586,
 0.996603910236679,
 0.998236862156483,
 0.998982143527128,
 0.999213606237084,
 0.999048392017777,
 0.998403878905059,
 0.996955573848852,
 0.993977951531375,
 0.987992351319893]

In [ ]:
table

,x,y,df(x)
0,"[-1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -...","[-1.07645464892126, 0.546159934812791, -0.8243...","[-267.925064211607, 121.823653191237, -272.138..."
1,"[-1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -...","[-0.810350221164158, 0.658754347153573, -0.700...","[-2.94426344350433, 298.735656013492, -217.266..."
2,"[-1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -...","[-0.831462898688653, 0.608354672118218, -0.635...","[-31.2594720058003, 227.427928816017, -158.931..."
3,"[-1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -...","[-0.919262672716396, 0.703747551776220, -0.658...","[-55.7938946555927, 295.996036128778, -199.841..."
4,"[-1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -1.2, 1.0, -...","[-0.933853718696140, 0.664062351949551, -0.642...","[-81.5719630458559, 245.493847009460, -185.124..."
...,...,...,...
798,"[-0.993264209878781, 0.996607331028612, 0.9982...","[-0.993261818772765, 0.996605317242838, 0.9982...","[0.000936506616710631, 0.00117671793321961, 0...."
799,"[-0.993261361769827, 0.996604806406364, 0.9982...","[-0.993261361769827, 0.996604806406364, 0.9982...","[0.000936506616710631, 0.00117671793321961, 0...."
800,"[-0.993261361769827, 0.996604806406364, 0.9982...","[-0.993262001982900, 0.996604001980436, 0.9982...","[0.000269716030402378, -0.000162599059706281, ..."
801,"[-0.993261361769827, 0.996604806406364, 0.9982...","[-0.993262714877388, 0.996603626815056, 0.9982...","[-0.000440559946468966, -0.000667923525782044,..."


In [ ]:
# Para n = 50
import numpy as np
import sympy as sym 


variaveis = list(sym.symbols("x:50"))

c = variaveis

def f(c):
  fo = 0
  for i in range(1,50):
     fo = fo + 100*(c[i] - c[i-1]**2)**2 + (1 - c[i-1])**2 
  return fo

x = []
for i in range(1,51):
    if (i%2 != 0):
        x.append(-1.2)
    else:
        x.append(1)  

eps = 1e-3
p  = Parametros(f(c),x,c,eps)
y,table = Davidon_Fletcher_Powell(p)

KeyboardInterrupt: ignored

In [ ]:
y

In [ ]:
table

Este trecho foi interrompido pois depois de aproximadamente 40 min, o gradiente começou a aumentar, se distanciando, portanto, da solução. Desse modo, não houve a convergência deste método usando 50 variáveis. Isso se deve à grande quantidade de vetores extensos e necessidade de manupulação destes para resolver o problema. 

**2.** Resolva:

Minimizar $(x_1 - x^3_2)^2 + 3(x_1 - x_2)^4$

In [ ]:
import numpy as np
import sympy as sym 
import pandas as pd
import math

x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
variaveis = [x1,x2]

c = variaveis

def fo(c):
     return (c[0] - c[1]**3)**2 + 3*(c[0] - c[1])**4

x = [1.2,1.5]

eps = 1e-3
nmax = 7
p  = Parametros(fo(c),x,c,eps)
y,table = Davidon_Fletcher_Powell(p)

In [ ]:
y

[1.05453004230805, 1.01789085244265]

In [ ]:
table

,x,y,df(x)
0,"[1.2, 1.5]","[1.26535873347338, 1.08487975154954]","[0.0475326522102890, 0.0107070255988813]"
1,"[1.21320466631612, 1.07585039918436]","[1.21320466631612, 1.07585039918436]","[0.0475326522102890, 0.0107070255988813]"
2,"[1.21320466631612, 1.07585039918436]","[1.19385242940156, 1.07149118743402]","[-0.0506546747373421, 0.228204703643416]"
3,"[1.17172646122159, 1.05388219082351]","[1.17172646122159, 1.05388219082351]","[-0.0506546747373421, 0.228204703643416]"
4,"[1.17172646122159, 1.05388219082351]","[1.17144866336849, 1.05513369973298]","[0.0124053367750883, 0.00275361755094361]"
5,"[1.12320180547598, 1.04187681579765]","[1.12320180547598, 1.04187681579765]","[0.0124053367750883, 0.00275361755094361]"
6,"[1.12320180547598, 1.04187681579765]","[1.12082641102867, 1.04134954854492]","[-0.0108203223147021, 0.0487749740716879]"
7,"[1.11387010795007, 1.03646728218220]","[1.11387010795007, 1.03646728218220]","[-0.0108203223147021, 0.0487749740716879]"
8,"[1.11387010795007, 1.03646728218220]","[1.11378004399109, 1.03687326521452]","[0.00352092457619746, 0.000790848090488658]"
9,"[1.08117662184621, 1.02736932370259]","[1.08117662184621, 1.02736932370259]","[0.00352092457619746, 0.000790848090488658]"


**3.** Resolva:



$2(x_1 - 2)^4 + (2x_1 - x_2)^2 = 4$


In [ ]:
import numpy as np
import sympy as sym 

x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
c = [x1,x2]

def fo(c):
     return (2*(c[0] - 2)**4 + (2*c[0] - c[1])**2 - 4)**2

x = [1.2,2]

eps = 1e-3
nmax = 7
p  = Parametros(fo(c),x,c,eps)
y,table = Davidon_Fletcher_Powell(p)

In [ ]:
y

[1.97585164252150, 3.95169228507291]

In [ ]:
table

,x,y,df(x)
0,"[1.2, 2.0]","[1.35045430325702, 2.04822253309520]","[-2.69247368424650, 8.40133714524466]"
1,"[1.58717567512133, 3.26408285714833]","[1.58717567512133, 3.26408285714833]","[-2.69247368424650, 8.40133714524466]"
2,"[1.58717567512133, 3.26408285714833]","[1.61350853538107, 3.18191632072678]","[2.22538489024428, 0.713194351729630]"
3,"[1.66334938968787, 3.21406511684388]","[1.66334938968787, 3.21406511684388]","[2.22538489024428, 0.713194351729630]"
4,"[1.66334938968787, 3.21406511684388]","[1.65467907947995, 3.21128644413196]","[-0.498097504414925, 1.55421796638029]"
5,"[1.76884028190128, 3.57833837067244]","[1.76884028190128, 3.57833837067244]","[-0.498097504414925, 1.55421796638029]"
6,"[1.76884028190128, 3.57833837067244]","[1.77844014230379, 3.54838384293424]","[0.423665595991962, 0.135776821850525]"
7,"[1.79211383206660, 3.55740885010328]","[1.79211383206660, 3.55740885010328]","[0.423665595991962, 0.135776821850525]"
8,"[1.79211383206660, 3.55740885010328]","[1.78939250127265, 3.55653671498091]","[-0.113956153248817, 0.355578378252645]"
9,"[1.87992678379448, 3.77612770095414]","[1.87992678379448, 3.77612770095414]","[-0.113956153248817, 0.355578378252645]"


**4** Resolva:

Minimizar $(x_1 - 3)^4 + (x_1 - 3x_2)^2$

In [ ]:
import numpy as np
import sympy as sym 

x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
c = [x1,x2]

def fo(c):
     return (c[0] - 3)**4 + (c[0] - 3*c[1])**2 

x = [1.5,2]

eps = 1e-3
p  = Parametros(fo(c),x,c,eps)
m,table = Davidon_Fletcher_Powell(p)

In [ ]:
m

[2.93741017755409, 0.979101208908746]

In [ ]:
table

,x,y,df(x)
0,"[1.5, 2.0]","[2.49073861653863, 0.811113660153645]","[-0.413506695085297, -0.344385816466160]"
1,"[2.59276775978705, 0.894972839719642]","[2.59276775978705, 0.894972839719642]","[-0.413506695085297, -0.344385816466160]"
2,"[2.59276775978705, 0.894972839719642]","[2.59183614644195, 0.894196952832449]","[-0.453506112200395, 0.544528272332393]"
3,"[2.74263279272075, 0.901051456174850]","[2.74263279272075, 0.901051456174850]","[-0.453506112200395, 0.544528272332393]"
4,"[2.74263279272075, 0.901051456174850]","[2.73579601838682, 0.909260423631364]","[-0.0577402132943634, -0.0480884849563665]"
5,"[2.76960744565758, 0.931544682153159]","[2.76960744565758, 0.931544682153159]","[-0.0577402132943634, -0.0480884849563665]"
6,"[2.76960744565758, 0.931544682153159]","[2.76453410742791, 0.927319392095910]","[-0.0870689972292755, 0.104544413158933]"
7,"[2.85458636521515, 0.946312279202572]","[2.85458636521515, 0.946312279202572]","[-0.0870689972292755, 0.104544413158933]"
8,"[2.85458636521515, 0.946312279202572]","[2.85149513186943, 0.950023946803551]","[-0.0102537418634556, -0.00853974875268904]"
9,"[2.85881779993772, 0.954846185400209]","[2.85881779993772, 0.954846185400209]","[-0.0102537418634556, -0.00853974875268904]"


**Conclusões**

O método Quasi-Newton, Davidon-Fletcher-Powell, consegue uma convergência mais rápida que o método do Gradiente Conjugado, Fletcher-Revees, sem tanta variação pela região de busca, levando em geral menos iterações com uma precisão razoável. 


O problema envolvendo 5 e 10 variáveis foi solucionado mais rapidamente pelo método Quasi-Newton, porém em ambos, o tempo levado para 50 variáveis era inviáveis e em algumas tentativas, o resultado divergia da solução, principalmente no caso do Gradiente Conjugado, ao qual apresenta bastante oscilação em torno da solução.

O método do Gradiente Conjugado obtém uma precisão muito alta e consegue a convergência, mesmo lenta, quando se trabalha com uma maior número de variáveis. Pode acontecer de chegar a oscilar muito em torno da resposta e não convergir quando os valores são muito distantes da solução ou a precisão requerida não for muito grande. 
